#             Assignment No- 3 Web Scraping

# 1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [36]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
driver.minimize_window()

In [38]:
try:
    #opening a amazon page on automated chrome browser
    driver.get("http://www.amazon.in/")
    #to take input from user requred for search
    item=input("\nEnter the item to search on amazon shopping website:\n")
    #opening web page to search for user entered item 
    search_box = driver.find_element(By.ID,"twotabsearchtextbox").send_keys(item)
    #click on the search button
    driver.maximize_window()
    search_button = driver.find_element(By.ID,"nav-search-submit-text").click()

except Exception as e:
    print(e)


Enter the item to search on amazon shopping website:
gitar


# 2.In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [39]:
try:
    driver.execute_script("window.scrollTo(0,500)")
   #for scraping
    Brand = []
    Prod_name = []
    Return_exchange = []
    Expected_delivary = []
    Availability = []
    product_url = []
    Price = []
    
except NoSuchElementException as e:
    print("Exception raised:",e)
    driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[3]/div/div/div/div/div/div/div[2]/div[2]/h2/a").click()
    

In [40]:
try:
    # for scraping all Product urls
    #scroll to pixel 800 of webpage
    prod_urls = []
    start=0
    end=3
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        for i in url[0:10]:
            prod_urls.append(i.get_attribute("href"))
    nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    nxt_button.click()
    time.sleep(2)

except NoSuchElementException as e:
    print(e)
    
#print(len(prod_urls)) #all urls number we print


In [43]:
for url in prod_urls[:10]:#loop for every gitar in list
    driver.get(url)
    time.sleep(2)
    try:
            with open('Product_details_intern34_ass3_Q2.csv','w',encoding='utf8',newline='') as f:
            #writer will write in file name 'Product_details_intern34_ass3_Q2.csv' object is created for stored in to thewriter variable name
                thewriter = writer(f)
                #this will be the column name lets say header
                header=['Brand Name','Price','Return/Exchange','Expected Delivery','Availability','Product URL']
                #writerwill write the row in csv file
                thewriter.writerow(header)
                
                # scraping Product name
                product_nametags = driver.find_element(By.XPATH,'//*[@id="titleSection"]//h1//span')
                Prod_name.append(product_nametags.text)
                #thewriter.writerow(Prod_name)
                
                #Scraping availability status of of product
                availabilitytags = driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
                Availability.append(availabilitytags.text)
                #thewriter.writerow(Availability)
                
                #scraping Product Price
                pricetags = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')
                Price.append(pricetags.text)
                #thewriter.writerow(Price)
                
                # scraping expected_delivary date from 3 pages first 10 products
                expected_delivarytags = driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
                Expected_delivary.append(expected_delivarytags.text)
                #thewriter.writerow(Expected_delivary)
                
                
                #scroll down to extract brands and return/exchang
                driver.execute_script("window.scrollTo(0,900)")
        
                #scraping Brand names 
                Brandtags = driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]//div//table//tbody//tr[1]//td[2]//span')
                Brand.append(Brandtags.text)
                #thewriter.writerow(Brand)
                
                # scraping return/exchange policy
                ReturnExchangetags =driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]//span//div[2]//a')
                Return_exchange.append(ReturnExchangetags.text)
                #thewriter.writerow(Return_exchange)
                
                #write the info into csv file
                infoList=[Brand,Price,Return_exchange,Expected_delivary,Availability,prod_urls]
                thewriter.writerow(infoList)
        
    except NoSuchElementException as e:
            Brand.append('-')
            Return_exchange.append('-')
            Prod_name.append('-')
            Expected_delivary.append('-')
            Price.append('-')
            prod_urls.append('-')
   
  

In [57]:
#Read the csv file 'Product_details_ass3_2.csv' and store the read_csv object int product_data variable using pd
data=pd.read_csv('Product_details_intern34_ass3_Q2.csv')
#for print prod_data_first3pages) the csv file data :---->
for i in data:
    print("\n",data[i])


 0    ['Medellin', 'Medellin', 'Wooden', 'Multi-colo...
Name: Brand Name, dtype: object

 0    ['2,399', '2,429', '799', '899', '2,090', '2,4...
Name: Price, dtype: object

 0    ['7 days Returnable', '7 days Replacement', '7...
Name: Return/Exchange, dtype: object

 0    ['Sunday, 25 December', 'Saturday, 24 December...
Name: Expected Delivery, dtype: object

 0    ['In stock.', 'In stock.', 'In stock.', 'In st...
Name: Availability, dtype: object

 0    ['https://www.amazon.in/sspa/click?ie=UTF8&spc...
Name: Product URL, dtype: object


In [58]:
#print(len(Brand)) #Print length of Brand name 
#for i in Brand:
   # print(i)
#for i in  Return_exchange:
 #   print(i)
#print(len(Return_exchange))
#print(len(Prod_name))
#print(len(Expected_delivary))
#print(len(Price))
#print(len(Availability))
#print(len(prod_urls))

In [59]:
#creating a data frame
import pandas as pd
df2= pd.DataFrame({"Brand":Brand[0:10],"Price(₹)":Price[0:10],"Return_exchange":Return_exchange[0:10],"Expected_delivary":Expected_delivary[0:10],"Availability":Availability[0:10],"prod_urls":prod_urls[0:10]})
df2



,Brand,Price(₹),Return_exchange,Expected_delivary,Availability,prod_urls
0,Medellin,"2,399",7 days Returnable,"Sunday, 25 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Medellin,"2,429",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Wooden,799,7 days Replacement,"Monday, 26 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Multi-coloured,899,7 days Replacement,"Sunday, 25 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Juârez,"2,090",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,JUAREZ,"2,445",7 days Replacement,"Sunday, 25 December",In stock.,https://www.amazon.in/Acoustic-Cutaway-RDS-Str...
6,JUAREZ,"2,514",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
7,BigPlayer,"9,664",7 days Replacement,"Saturday, 24 December",Only 2 left in stock.,https://www.amazon.in/BigPlayer-gitar-listrik-...
8,JUAREZ,"6,864",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/JUAREZ-Arpeggio-Guitar-J...
9,YAMAHA,"11,390",7 days Replacement,"Tuesday, 27 December",Only 1 left in stock.,https://www.amazon.in/Yamaha-Acoustic-Guitar-D...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [3]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
driver.minimize_window()

In [4]:
try:
    #opening a amazon page on automated chrome browser
    driver.get("http://www.images.google.com/")
    driver.maximize_window()
    item=input("\nEnter the item to search on google images\n")
    #opening web page to search for user entered item 
    search_box = driver.find_element(By.XPATH,'//input[@class="gLFyf"]').send_keys(item)
    #click on the search button
    driver.maximize_window()
    search_button = driver.find_element(By.XPATH,'//button[@class="Tg7LZd"]').click()
except Exception as e:
    print(e)


Enter the item to search on google images
cars


In [7]:
import requests
for _ in range(0,20):
    driver.execute_script("window.scrollBy(0,10)")

images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
i=1
for i in range(len(img_urls)):
    if i>10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\Users\Atharv\Desktop\IMAGE\image"+str(i)+".jpg","wb")
    file.write(response.content)


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [183]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [190]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [191]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [192]:
try:
    #opening a flipkart page on automated chrome browser
    driver.get("http://www.flipkart.com/")
    
    #closing login popup
    driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]').click()
    
    #to take input from user need to search
    item=input("\nEnter the item to search on flipkart for a smartphone e.g.: Oneplus Nord, pixel 4A) \n")
    #opening web page to search for user entered item
    search_box = driver.find_element(By.XPATH,'//input[@class="_3704LK"]').send_keys(item)
    #click on the search button
    search_button = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]').click() 
    driver.maximize_window()

except NoSuchException as e:
    driver.get("http://www.flipkart.com/")
    item=input("\nEnter the item to search on flipkart for a smartphone (e.g.: Oneplus Nord, pixel 4A) \n")
    search_box = driver.find_element(By.XPATH,'//input[@class="_3704LK"]').send_keys(item)
    print(e)
    


Enter the item to search on flipkart for a smartphone e.g.: Oneplus Nord, pixel 4A) 
pixel 4A


In [194]:
try:
    # for scrolling the page 
    driver.execute_script("window.scrollTo(0,100)")
    # for scraping all Product urls
    start=0
    end=1
    prod_urls = []  #list to store all product url
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
        for i in url:
            prod_urls.append(i.get_attribute("href"))
            #nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'
            #nxt_button.click()
            time.sleep(2)
        #print number of items present on 1st page     
        print(len(prod_urls))
                        
                       
except Exception as e:
    print(e)
    

24


In [195]:
Modelname = []
Phonename = []
Storage_ROM = []
Storage_RAM = []
Primary_Camera = []
Secondary_Camera = []
Display_Size =[]
Battery_Capacity = []
Price = []
Color = []

#for scraping each items details
for url in prod_urls[0:24]:
    driver.get(url)
    time.sleep(5)
    try:
        with open('Smartphone_intern34_ass3_Q2.csv','w',encoding='utf8',newline='') as f:
            #writer will write in file name 'Product_details_intern34_ass3_Q2.csv' object is created for stored in to thewriter variable name
            thewriter = writer(f)
            #this will be the column name lets say header
            header=["Phonename","Price(₹)","Modelname","Color","Display_Size","Storage_ROM","Storage_RAM","Primary_Camera","Secondary_Camera","Battery","prod_urls"]
            thewriter.writerow(header)
            
            #Scroll up    
            js = 'window.scrollTo(0, document.documentElement.scrollHeight)'
            driver.execute_script(js)
            time.sleep(3)
            #scraping product name,Price
            
            Phonenametags = driver.find_element(By.XPATH,'//*[@class="yhB1nd"]//span')
            Phonename.append(Phonenametags.text[0:10])
            
            Pricetags =driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
            Price.append(Pricetags.text.replace("₹",""))
            
            #scroll down
            driver.execute_script("window.scrollTo(0,2200)")
            #to click on real More
            driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]').click()
            
            #for scraping model name
            Modelnametags = driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]//tr[3]//td[2]')
            Modelname.append(Modelnametags.text)
            
            #for scraping color
            Colortags = driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]//tr[4]//td[2]')
            Color.append(Colortags.text)
            
            #driver.execute_script("window.scrollTo(0,500)")
            
            #for scraping Display size
            displaytags = driver.find_element(By.XPATH,'//div[2][@class="_3k-BhJ"]//tr[1]//td[2]')
            Display_Size.append(displaytags.text)
            
            #scroll down
            driver.execute_script("window.scrollTo(0,1000)")
            
            #for scraping storage ROM
            ROMtags = driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]//tr[1]//td[2]')
            Storage_ROM.append(ROMtags.text)
            
            #for scraping RAM
            RAMtags = driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]//tr[2]//td[2]')
            Storage_RAM.append(RAMtags.text)
            
            # for scraping Primary camera
            rearcamera_tags = driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]//tr[2]//td[2]')
            Primary_Camera.append(rearcamera_tags.text)
            
            #for scraping Secondary camera
            frontcamera_tags = driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]//tr[5]//td[2]')
            Secondary_Camera.append(frontcamera_tags.text)
            
            #scroll down
            driver.execute_script("window.scrollTo(0,900)")
        
            #for scraping Battery Capacity
            batterytags = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr/td[2]/ul/li')
            Battery_Capacity.append(batterytags.text)
            
            #Scroll up    
            js = 'window.scrollTo(0, document.documentElement.scrollHeight)'
            driver.execute_script(js)
            time.sleep(3)
            
            #write the info into csv file
            Smartphone_infoList=[Phonename,Price,Modelname,Color,Display_Size,Storage_ROM,Storage_RAM,Primary_Camera,Secondary_Camera,Battery_Capacity,prod_urls]
            thewriter.writerow(Smartphone_infoList)
            
            
    except Exception as e:
        Phonename.append('-')
        Price.append('-')
        Modelname.append('-')
        Color.append('-')
        Display_Size.append('-')
        Storage_ROM.append('-')
        Storage_RAM.append('-')
        Primary_Camera.append('-')
        Secondary_Camera.append('-')
        Battery_Capacity.append('-')
        prod_urls.append('-')

In [196]:
print(Phonename)
print(Price)
print(Modelname)
print(Color)
print(Display_Size)
print(Storage_ROM)
print(Storage_RAM)
print(Primary_Camera)
print(Secondary_Camera)
print(Battery_Capacity)

['Google Pix', '-', 'realme 10 ', 'realme 10 ', 'Redmi Note', '-', 'realme 10 ', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'vivo T1 44', 'vivo T1 44', 'realme 10 ', 'realme 10 ', 'realme 10 ', 'realme 10 ', '-', 'vivo T1 44', 'vivo T1 44', 'vivo T1 44', 'realme C33', 'realme C33', 'realme C33', 'realme C33']
['31,999', '-', '19,999', '19,999', '12,489', '-', '18,999', '5,999', '5,999', '5,999', '6,999', '6,999', '6,999', '14,499', '14,499', '24,999', '25,999', '25,999', '24,999', '-', '14,499', '15,999', '15,999', '8,999', '8,999', '9,999', '9,999']
['Pixel 4a', '-', '10 Pro 5G', '10 Pro 5G', '-', '10 Pro 5G', 'C30', 'C30', 'C30', 'C30', 'C30', 'C30', 'T1 44W', 'T1 44W', '10 Pro+ 5G', '10 Pro+ 5G', '10 Pro+ 5G', '10 Pro+ 5G', '-', 'T1 44W', 'T1 44W', 'T1 44W', 'C33', 'C33', 'C33', 'C33']
['Just Black', '-', 'Hyperspace', 'Nebula Blue', '-', 'Dark Matter', 'Denim Black', 'Bamboo Green', 'Lake Blue', 'Lake Blue', 'Bamboo Green', 'Denim Black', 'S

In [197]:
#Read the csv file 'Product_details_ass3_2.csv' and store the read_csv object int product_data variable using pd
data=pd.read_csv('Smartphone_intern34_ass3_Q2.csv')
#for print prod_data_first3pages) the csv file data :---->
for i in data:
    print("\n",data[i])


 0    ['Google Pix', '-', 'realme 10 ', 'realme 10 '...
Name: Phonename, dtype: object

 0    ['31,999', '-', '19,999', '19,999', '12,489', ...
Name: Price(₹), dtype: object

 0    ['Pixel 4a', '-', '10 Pro 5G', '10 Pro 5G', '-...
Name: Modelname, dtype: object

 0    ['Just Black', '-', 'Hyperspace', 'Nebula Blue...
Name: Color, dtype: object

 0    ['14.76 cm (5.81 inch)', '-', '17.07 cm (6.72 ...
Name: Display_Size, dtype: object

 0    ['128 GB', '-', '128 GB', '128 GB', '-', '128 ...
Name: Storage_ROM, dtype: object

 0    ['6 GB', '-', '8 GB', '8 GB', '-', '6 GB', '2 ...
Name: Storage_RAM, dtype: object

 0    ['12.2MP Rear Camera', '-', '108MP + 2MP', '10...
Name: Primary_Camera, dtype: object

 0    ['8MP Front Camera', '-', '16MP Front Camera',...
Name: Secondary_Camera, dtype: object

 0    ['-', '5000 mAh', '5000 mAh', '-', '5000 mAh',...
Name: Battery, dtype: object

 0    ['https://www.flipkart.com/google-pixel-4a-jus...
Name: prod_urls, dtype: object


In [199]:
#creating a data frame
import pandas as pd
df4= pd.DataFrame({"Phonename": Phonename[0:24],"Price(₹)":Price[0:24],"Modelname":Modelname[0:24],"Color":Color[0:24],"Display_Size":Display_Size[0:24],"Storage_ROM":Storage_ROM[0:24],"Storage_RAM":Storage_RAM[0:24],"Primary_Camera":Primary_Camera[0:24],"Secondary_Camera":Secondary_Camera[0:24],"BatteryCapacity":Battery_Capacity[0:24],"prod_urls":prod_urls[0:24]})
df4


,Phonename,Price(₹),Modelname,Color,Display_Size,Storage_ROM,Storage_RAM,Primary_Camera,Secondary_Camera,BatteryCapacity,prod_urls
0,Google Pix,"31,999",Pixel 4a,Just Black,14.76 cm (5.81 inch),128 GB,6 GB,12.2MP Rear Camera,8MP Front Camera,-,https://www.flipkart.com/google-pixel-4a-just-...
1,-,-,-,-,-,-,-,-,-,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-hype...
2,realme 10,"19,999",10 Pro 5G,Hyperspace,17.07 cm (6.72 inch),128 GB,8 GB,108MP + 2MP,16MP Front Camera,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-nebu...
3,realme 10,"19,999",10 Pro 5G,Nebula Blue,17.07 cm (6.72 inch),128 GB,8 GB,108MP + 2MP,16MP Front Camera,-,https://www.flipkart.com/redmi-note-11-space-b...
4,Redmi Note,"12,489",-,-,-,-,-,-,-,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-dark...
5,-,-,10 Pro 5G,Dark Matter,17.07 cm (6.72 inch),128 GB,6 GB,108MP + 2MP,16MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-denim-blac...
6,realme 10,"18,999",C30,Denim Black,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-bamboo-gre...
7,realme C30,"5,999",C30,Bamboo Green,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-lake-blue-...
8,realme C30,"5,999",C30,Lake Blue,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-lake-blue-...
9,realme C30,"5,999",C30,Lake Blue,16.51 cm (6.5 inch),32 GB,3 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-bamboo-gre...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [224]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [225]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [325]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [4]:
try:
    #opening a flipkart page on automated chrome browser
    driver.get("https://www.google.com/maps")
    #to take input from user need to search
    item=input("\nlocation a city searched on google maps:")
    #opening web page to search for user entered item
    search_box = driver.find_element(By.XPATH,'//input[@class="searchboxinput xiQnY"]').send_keys(item)
    #click on the search button
    search_button = driver.find_element(By.XPATH,'//button[@class="mL3xi"]').click() 
    driver.maximize_window()
    

except NoSuchException as e:
    print(e)


location a city searched on google maps:London


In [6]:
import re
lar_lng = []
lar = []
long = []
try:
    url_string = driver.current_url
    print("URL extracted:",url_string)
    lar_lng = re.findall(r'@(.*)data',url_string)
    #print(lar_lng)
    #to separate list 
    new=[]
    for i in lar_lng:
        for j in lar_lng:
            new.append(j.split(','[0]))
    #print(new)
    #to retrieve and print latitude,longitude of search city
    lar = new[0][0]
    long = new[0][1]
    print("latitude of",item,"is",lar)
    print("longitude of",item,"is",long)
except Exception as e:
    print(e)

URL extracted: https://www.google.com/maps/place/London,+UK/@51.6173162,-0.0729103,9z/data=!4m5!3m4!1s0x47d8a00baf21de75:0x52963a5addd52a99!8m2!3d51.5072178!4d-0.1275862
latitude of London is 51.6173162
longitude of London is -0.0729103


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [200]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [201]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [202]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [203]:
try:
    
    # opening the trak page on automated chrome browser
    driver.get("https://trak.in/tags/business/2022/09/28/indian-startups-receive-rs-7900-crore-vc-funding-in-august-2022-vc-fundings-up-by-9-7/#:~:text=Indian%20Startups%20Receive%20Rs%207900,Business%20of%20Tech%2C%20Mobile%20%26%20Startups")
    driver.maximize_window()
    #close ad popup
    time.sleep(3)
    #driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[2]/div/div/div[2]/div/div/div[3]').click()
    #click on funding deals
    
    fundstags =driver.find_element(By.XPATH,'//li[@class="menu-have-icon menu-icon-type-fontawesome menu-item menu-item-type-post_type menu-item-object-page better-anim-fade menu-item-51510"]//a')
    fundstags.click()
    
    #close popup
    #driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[2]/').click()
    #scroll down
    #driver.execute_script("window.scrollTo(0,400)")

except NoSuchElementException as e:
    driver.get("https://trak.in/tags/business/2022/09/28/indian-startups-receive-rs-7900-crore-vc-funding-in-august-2022-vc-fundings-up-by-9-7/#:~:text=Indian%20Startups%20Receive%20Rs%207900,Business%20of%20Tech%2C%20Mobile%20%26%20Startups")
    print(e)
    
    

In [218]:
#scroll down
driver.execute_script("window.scrollTo(0,400)") #we dont want Aril 2021 details so just pass down we scroll

In [219]:
#Scraping March,2021
#scroll down
driver.execute_script("window.scrollTo(400,900)")

inforow1 = []
inforow2 = []
inforow3 = []
inforow4 = []
inforow5 = []
#table = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]')
#for i in table:

row1tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[1]')
for j in row1tag:
    row1 = j.text.split(" ")
    inforow1.append(row1)


row2tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[2]')
for j in row2tag:
    row2 = j.text.split(" ")
    inforow2.append(row2)

row3tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[3]')
for j in row3tag:
    row3 = j.text.split(" ")
    inforow3.append(row3)

row4tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[4]')
for j in row4tag:
    row4 = j.text.split(" ")
    inforow4.append(row4)

row5tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[5]')
for j in row5tag:
    row5 = j.text.split(" ")
    inforow5.append(row5)

In [207]:
serial_no = [inforow1[1][0],inforow2[1][0],inforow3[1][0],inforow4[1][0],inforow5[1][0]]
serial_no


['1', '2', '3', '4', '5']

In [208]:
date = [inforow1[1][1],inforow2[1][1],inforow3[1][1],inforow4[1][1],inforow5[1][1]]
date

['04/03/2021', '31/03/2021', '30/03/2021', '30/03/2021', '23/03/2021']

In [209]:
startup_name = [inforow1[1][2],inforow2[1][2],inforow3[1][2],inforow4[1][2],inforow5[1][2]]
startup_name

['DealShare', 'Uniphore', 'Dunzo', 'BYJU’S', 'SkilloVilla']

In [210]:
industry_vertical = [inforow1[1][3],inforow2[1][3],inforow3[1][3],inforow4[1][3],inforow5[1][3]]
industry_vertical

['E-commerce', 'Technology', 'E-commerce', 'Edu-tech', 'Edu-tech']

In [211]:
sub_vertical = [inforow1[1][4]+" "+inforow1[1][5]+" "+inforow1[1][6],inforow2[1][4]+" "+inforow2[1][5]+" "+inforow2[1][6]+" "+inforow2[1][7],inforow3[1][4]+" "+inforow3[1][5]+" "+inforow3[1][6],inforow4[1][4]+" "+inforow4[1][5],inforow5[1][4]+" "+inforow5[1][5]+" "+inforow5[1][6]+" "+inforow5[1][7]]
sub_vertical

['Online shopping platform',
 'Conversational Service Automation (CSA)',
 'Hyper-local delivery app',
 'Online tutoring',
 'Career and job-oriented upskilling.']

In [212]:
city_location = [inforow1[1][7]+inforow1[1][8],inforow2[1][8]+inforow2[1][9],inforow3[1][7],inforow4[1][6],inforow5[1][8]]
city_location

['Jaipur,Rajasthan', 'PaloAlto', 'Bengaluru', 'Bengaluru', 'Bengaluru']

In [213]:
investors_name = [inforow1[1][9]+inforow1[1][10],inforow2[1][10]+inforow2[1][11]+inforow2[1][12],inforow3[1][8]+inforow3[1][9]+inforow3[1][10]+inforow3[1][11],inforow4[1][7]+inforow4[1][8]+inforow4[1][9]+inforow4[1][10]+inforow4[1][11]+inforow4[1][12]+inforow4[1][13],inforow5[1][9]+inforow5[1][10]+inforow5[1][11]]
investors_name

['InnovenCapital',
 'SorensonCapitalPartners',
 'KrishtalAdvisorsPteLtd',
 'MCGlobalEdtech,BCapital,Baron,others',
 'TitanCapital,others']

In [214]:
investmenType = [inforow1[1][11]+inforow1[1][12],inforow2[1][13]+inforow2[1][14],inforow3[1][12]+inforow3[1][13],inforow4[1][14]+inforow4[1][15],inforow5[1][12]]
investmenType

['DebtFinancing', 'SeriesD', 'SeriesE', 'SeriesF', 'Seed']

In [215]:
amount_in_USD = [inforow1[1][13],inforow2[1][15],inforow3[1][14],inforow4[1][16],inforow5[1][13]]
amount_in_USD

['250,000,000', '140,000,000', '8,000,000', '460,000,000', '300,000,000']

In [216]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)

Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-56-name"]')
for month in Monthtags:
    m = month.text
print(m)
    
rowstags = driver.find_elements(By.XPATH,'//tr[@class="row-1 odd"]')
header = []
#heading = []
for i in rowstags:
    title = i.text
    heading.append(title.split(" "[0]))

title1 = header[0:7]
title2 = header[8:25]
title3 = header[26:38] 
title4 = header[39:58]
title5 = header[59:71]
title6 = header[72:87]
title7 = header[88:104]
title8 = header[104:120]
title9 = header[120:]
heading = [title1,title2,title3,title4,title5,title6,title7,title8,title9]
#print(heading)
info_March_2021 = []
info_March_2021 = [inforow1[0],inforow2[0],inforow3[0],inforow4[0],inforow5[0]]
#creating dataframe ---- info_April_2021
#print("-----------------------------------------------------------------------")
df_March_2021 = pd.DataFrame({'Sr. No.':serial_no,
 'Date (dd/mm/yyyy)':date,
 'Startup Name':startup_name,
 'Industry / Vertical':industry_vertical,
 'Sub-Vertical':sub_vertical,
 'City / Location':city_location,
 "Investors' Name    ":investors_name,
 'Investment Type ':investmenType,
 'Amount (In USD)':amount_in_USD})
df_March_2021


Indian Startup Funding & Investment Chart [2021]
March, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur,Rajasthan",InnovenCapital,DebtFinancing,"250,000,000"
1,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),PaloAlto,SorensonCapitalPartners,SeriesD,"140,000,000"
2,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,KrishtalAdvisorsPteLtd,SeriesE,"8,000,000"
3,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MCGlobalEdtech,BCapital,Baron,others",SeriesF,"460,000,000"
4,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"TitanCapital,others",Seed,"300,000,000"


In [221]:
#Scraping Feb,2021
#scroll down
driver.execute_script("window.scrollTo(900,1500)")

inforow1 = []
inforow2 = []
inforow3 = []
inforow4 = []
inforow5 = []
#table = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]')
#for i in table:

row1tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[1]')
for j in row1tag:
    row1 = j.text.split(" ")
    inforow1.append(row1)


row2tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[2]')
for j in row2tag:
    row2 = j.text.split(" ")
    inforow2.append(row2)

row3tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[3]')
for j in row3tag:
    row3 = j.text.split(" ")
    inforow3.append(row3)

row4tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[4]')
for j in row4tag:
    row4 = j.text.split(" ")
    inforow4.append(row4)

row5tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[5]')
for j in row5tag:
    row5 = j.text.split(" ")
    inforow5.append(row5)

In [222]:
serial_no = [inforow1[2][0],inforow2[2][0],inforow3[2][0],inforow4[2][0],inforow5[2][0]]
serial_no

['1', '2', '3', '4', '5']

In [223]:
date = [inforow1[2][1],inforow2[2][1],inforow3[2][1],inforow4[2][1],inforow5[2][1]]
date

['11/02/2021', '22/02/2021', '19/02/2021', '17/02/2021', '15/02/2021']

In [224]:
startup_name = [inforow1[2][2],inforow2[2][2],inforow3[2][2],inforow4[2][2],inforow5[2][2]]
startup_name

['Doubtnut', 'Zomato', 'Fingerlix', 'Zolve', 'KreditBee']

In [226]:
industry_vertical = [inforow1[2][3]+inforow1[2][4],inforow2[2][3],inforow3[2][3],inforow4[2][3],inforow5[2][3]]
industry_vertical

['EduTech', 'Hospitality', 'Hospitality', 'FinTech', 'Finance']

In [236]:
sub_vertical = [inforow1[2][5]+" " +inforow1[2][6],inforow2[2][4]+" "+inforow2[2][5]+" "+inforow2[2][6]+" "+inforow2[2][7],inforow3[2][4]+" "+inforow3[2][5]+" "+inforow3[2][6]+" "+inforow3[2][7],inforow4[2][4]+" "+inforow4[2][5]+" "+inforow4[2][6],inforow5[2][4]+" "+inforow5[2][5]+" "+inforow5[2][6]]
sub_vertical

['E-Learning Platform',
 'Online Food Delivery Platform',
 'Semi-cooked food delivery app',
 'Global Neobank Venture',
 'Digital lending platform']

In [243]:
city_location = [inforow1[2][7],inforow2[2][8],inforow3[2][8],inforow4[2][7],inforow5[2][7]]
city_location

['Gurgaon', 'Gurgaon', 'Mumbai', 'Mumbai', 'Bengaluru']

In [263]:
investors_name = [inforow1[2][8]+" "+inforow1[2][9]+inforow1[2][10]+inforow1[2][11]+inforow1[2][12]+inforow1[2][13]+" "+inforow1[2][14]+" "+inforow1[2][15]+" "+inforow1[2][16],inforow2[2][9]+inforow2[2][10]+inforow2[2][11],inforow3[2][9]+inforow3[2][10]+inforow3[2][11]+inforow3[2][12]+inforow3[2][13]+inforow3[2][14],inforow4[2][8]+inforow4[2][9]+inforow4[2][10]+inforow4[2][11]+inforow4[2][12]+inforow4[2][13],inforow5[2][8]+inforow5[2][9]+inforow5[2][10]+inforow5[2][11]+inforow5[2][12]+inforow5[2][13]+inforow5[2][14]+inforow5[2][15]+inforow5[2][16]]
investors_name

['SIG Global,SequoiaCapital,WaterBridgeVentures and ON Mauritius',
 'TigerGlobal,Kora',
 'RhodiumTrust,AccelPartnersandSwiggy',
 'AccelPartnersandLightspeedVenturePartners',
 'AzimPremji’sPremjiInvestandSouthKorea’sMiraeAssetVenture']

In [277]:
investmenType = [inforow1[2][17]+inforow1[2][18],inforow2[2][12],inforow3[2][15]+inforow3[2][16],inforow4[2][14],inforow5[2][17]+inforow5[2][18]]
investmenType

['SeriesB', 'Venture', 'SeriesC', 'Seed', 'SeriesC']

In [284]:
amount_in_USD = [inforow1[2][19],inforow2[2][13],inforow3[2][17],inforow4[2][15],inforow5[2][19]]
amount_in_USD

['2,500,000', '250,000,000', '2,747,045.20', '1,50,00,000', '75,000,000']

In [288]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)

Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-55-name"]')
for month in Monthtags:
    m = month.text
print(m)
    
rowstags = driver.find_elements(By.XPATH,'//tr[@class="row-1 odd"]')
header = []
#heading = []
for i in rowstags:
    title = i.text
    heading.append(title.split(" "[0]))

title1 = header[0:7]
title2 = header[8:25]
title3 = header[26:38] 
title4 = header[39:58]
title5 = header[59:71]
title6 = header[72:87]
title7 = header[88:104]
title8 = header[104:120]
title9 = header[120:]
heading = [title1,title2,title3,title4,title5,title6,title7,title8,title9]
#print(heading)
info_Feb_2021 = []
info_Feb_2021 = [inforow1[0],inforow2[0],inforow3[0],inforow4[0],inforow5[0]]
#creating dataframe ---- info_April_2021
#print("-----------------------------------------------------------------------")
df_Feb_2021 = pd.DataFrame({'Sr. No.':serial_no,
 'Date (dd/mm/yyyy)':date,
 'Startup Name':startup_name,
 'Industry / Vertical':industry_vertical,
 'Sub-Vertical':sub_vertical,
 'City / Location':city_location,
 "Investors' Name    ":investors_name,
 'Investment Type ':investmenType,
 'Amount (In USD)':amount_in_USD})
df_Feb_2021


Indian Startup Funding & Investment Chart [2021]
February, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,11/02/2021,Doubtnut,EduTech,Tech E-Learning Platform,Gurgaon,"SIG Global,SequoiaCapital,WaterBridgeVentures ...",SeriesB,"2,500,000"
1,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"TigerGlobal,Kora",Venture,"250,000,000"
2,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery,Mumbai,"RhodiumTrust,AccelPartnersandSwiggy",SeriesC,"2,747,045.20"
3,4,17/02/2021,Zolve,FinTech,Global Neobank,Mumbai,AccelPartnersandLightspeedVenturePartners,Seed,"1,50,00,000"
4,5,15/02/2021,KreditBee,Finance,Digital lending platform Bengaluru,Bengaluru,AzimPremji’sPremjiInvestandSouthKorea’sMiraeAs...,SeriesC,"75,000,000"


In [297]:
#for scraping Jan 2021 Data

#scroll down
driver.execute_script("window.scrollTo(1500,2200)")

inforow1 = []
inforow2 = []
inforow3 = []
inforow4 = []
inforow5 = []
#table = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]')
#for i in table:

row1tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[1]')
for j in row1tag:
    row1 = j.text.split(" ")
    inforow1.append(row1)


row2tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[2]')
for j in row2tag:
    row2 = j.text.split(" ")
    inforow2.append(row2)

row3tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[3]')
for j in row3tag:
    row3 = j.text.split(" ")
    inforow3.append(row3)

row4tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[4]')
for j in row4tag:
    row4 = j.text.split(" ")
    inforow4.append(row4)

row5tag = driver.find_elements(By.XPATH,'//tbody[@class="row-hover"]//tr[5]')
for j in row5tag:
    row5 = j.text.split(" ")
    inforow5.append(row5)


In [298]:
serial_no = [inforow1[3][0],inforow2[3][0],inforow3[3][0],inforow4[3][0],inforow5[3][0]]
serial_no

['1', '2', '3', '4', '5']

In [299]:
date = [inforow1[3][1],inforow2[3][1],inforow3[3][1],inforow4[3][1],inforow5[3][1]]
date

['15/01/2021', '28/01/2021', '19/01/2021', '19/01/2021', '18/01/2021']

In [302]:
startup_name = [inforow1[3][2]+inforow1[3][3],inforow2[3][2]+inforow2[3][3]+inforow2[3][4],inforow3[3][2],inforow4[3][2],inforow5[3][2]]
startup_name

['DigitInsurance', 'BombayShavingCompany', 'DeHaat', 'Darwinbox', 'mfine']

In [311]:
industry_vertical = [inforow1[3][4]+inforow1[3][5],inforow2[3][5]+inforow2[3][6]+inforow2[3][7],inforow3[3][3]+inforow3[3][4],inforow4[3][3],inforow5[3][3]+inforow5[3][4]+inforow5[3][5]]
industry_vertical

['FinancialServices',
 'ConsumerGoodsCompany',
 'AgriTechStartup',
 'SaaS',
 'HealthTechStartup']

In [322]:
sub_vertical = [inforow1[3][4]+" "+inforow1[3][5],inforow2[3][5]+" "+inforow2[3][6]+" "+inforow2[3][7],inforow3[3][5]+" "+inforow3[3][6]+inforow3[3][7]+inforow3[3][8]+inforow3[3][9]+inforow3[3][10]+inforow3[3][11],inforow4[3][4]+" "+inforow4[3][5],inforow5[3][6]+" "+inforow5[3][7]+" "+inforow5[3][8]+" "+inforow5[3][9]]
sub_vertical

['Financial Services',
 'Consumer Goods Company',
 'online marketplaceforfarmproductsandservices',
 'HR Tech',
 'AI-powered telemedicine mobile app']

In [329]:
city_location = [inforow1[3][8],inforow2[3][15]+" "+inforow2[3][16],inforow3[3][12],inforow4[3][6],inforow5[3][10]]
city_location

['Bengaluru', 'New Delhi', 'Patna', 'Mumbai', 'Bengaluru']

In [354]:
investors_name = [inforow1[3][9]+inforow1[3][10]+inforow1[3][11]+inforow1[3][12]+inforow1[3][13]+inforow1[3][14]+inforow1[3][15],inforow2[3][17]+inforow2[3][18],inforow3[3][13]+inforow3[3][14],inforow4[3][7]+inforow4[3][8],inforow5[3][11]+inforow5[3][12]+inforow5[3][13]]
investors_name

['A91Partners,FaeringCapital,TVSCapitalFunds',
 'ReckittBenckiser',
 'ProsusVentures',
 'SalesforceVentures',
 'HeritasCapitalManagement']

In [367]:
investmenType = [inforow1[3][16],inforow2[3][19],inforow3[3][15]+inforow3[3][16],inforow4[3][9],inforow5[3][14]+inforow5[3][15]]
investmenType

['Venture', 'Venture', 'SeriesC', 'Seed', 'VentureRound']

In [374]:
amount_in_USD = [inforow1[3][17],inforow2[3][20],inforow3[3][17],inforow4[3][10],inforow5[3][16]]
amount_in_USD

['1,80,00,000', '6,172,258.50', '30,000,000', '15,000,000', '16,000,000']

In [376]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)

Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-54-name"]')
for month in Monthtags:
    m = month.text
print(m)
    
rowstags = driver.find_elements(By.XPATH,'//tr[@class="row-1 odd"]')
header = []
#heading = []
for i in rowstags:
    title = i.text
    heading.append(title.split(" "[0]))

title1 = header[0:7]
title2 = header[8:25]
title3 = header[26:38] 
title4 = header[39:58]
title5 = header[59:71]
title6 = header[72:87]
title7 = header[88:104]
title8 = header[104:120]
title9 = header[120:]
heading = [title1,title2,title3,title4,title5,title6,title7,title8,title9]
#print(heading)
info_Jan_2021 = []
info_Jan_2021 = [inforow1[0],inforow2[0],inforow3[0],inforow4[0],inforow5[0]]
#creating dataframe ---- info_April_2021
#print("-----------------------------------------------------------------------")
df_Jan_2021 = pd.DataFrame({'Sr. No.':serial_no,
 'Date (dd/mm/yyyy)':date,
 'Startup Name':startup_name,
 'Industry / Vertical':industry_vertical,
 'Sub-Vertical':sub_vertical,
 'City / Location':city_location,
 "Investors' Name    ":investors_name,
 'Investment Type ':investmenType,
 'Amount (In USD)':amount_in_USD})
df_Jan_2021

Indian Startup Funding & Investment Chart [2021]
January, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/01/2021,DigitInsurance,FinancialServices,Financial Services,Bengaluru,"A91Partners,FaeringCapital,TVSCapitalFunds",Venture,"1,80,00,000"
1,2,28/01/2021,BombayShavingCompany,ConsumerGoodsCompany,Consumer Goods Company,New Delhi,ReckittBenckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTechStartup,online marketplaceforfarmproductsandservices,Patna,ProsusVentures,SeriesC,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,SalesforceVentures,Seed,"15,000,000"
4,5,18/01/2021,mfine,HealthTechStartup,AI-powered telemedicine mobile app,Bengaluru,HeritasCapitalManagement,VentureRound,"16,000,000"


In [379]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)

#Print Jan2021 month
Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-54-name"]')
for month in Monthtags:
    m = month.text
print(m)
#Jan21 details
df_Jan_2021


Indian Startup Funding & Investment Chart [2021]
January, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,15/01/2021,DigitInsurance,FinancialServices,Financial Services,Bengaluru,"A91Partners,FaeringCapital,TVSCapitalFunds",Venture,"1,80,00,000"
1,2,28/01/2021,BombayShavingCompany,ConsumerGoodsCompany,Consumer Goods Company,New Delhi,ReckittBenckiser,Venture,"6,172,258.50"
2,3,19/01/2021,DeHaat,AgriTechStartup,online marketplaceforfarmproductsandservices,Patna,ProsusVentures,SeriesC,"30,000,000"
3,4,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,SalesforceVentures,Seed,"15,000,000"
4,5,18/01/2021,mfine,HealthTechStartup,AI-powered telemedicine mobile app,Bengaluru,HeritasCapitalManagement,VentureRound,"16,000,000"


In [381]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)
Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-55-name"]')
for month in Monthtags:
    m = month.text
print(m)
#Feb21 details
df_Feb_2021

Indian Startup Funding & Investment Chart [2021]
February, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,11/02/2021,Doubtnut,EduTech,Tech E-Learning Platform,Gurgaon,"SIG Global,SequoiaCapital,WaterBridgeVentures ...",SeriesB,"2,500,000"
1,2,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"TigerGlobal,Kora",Venture,"250,000,000"
2,3,19/02/2021,Fingerlix,Hospitality,Semi-cooked food delivery,Mumbai,"RhodiumTrust,AccelPartnersandSwiggy",SeriesC,"2,747,045.20"
3,4,17/02/2021,Zolve,FinTech,Global Neobank,Mumbai,AccelPartnersandLightspeedVenturePartners,Seed,"1,50,00,000"
4,5,15/02/2021,KreditBee,Finance,Digital lending platform Bengaluru,Bengaluru,AzimPremji’sPremjiInvestandSouthKorea’sMiraeAs...,SeriesC,"75,000,000"


In [383]:
titletag = driver.find_elements(By.XPATH,'/html/body/div[1]/main/div/div/div/div[2]/div[1]/div/div/div/div[1]/div/h2[1]/span')
for i in titletag:
    title = i.text
print(title)
Monthtags = driver.find_elements(By.XPATH,'//h2[@id="tablepress-56-name"]')
for month in Monthtags:
    m = month.text
print(m)
#March21 details
df_March_2021

Indian Startup Funding & Investment Chart [2021]
March, 2021


,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry / Vertical,Sub-Vertical,City / Location,Investors' Name,Investment Type,Amount (In USD)
0,1,04/03/2021,DealShare,E-commerce,Online shopping platform,"Jaipur,Rajasthan",InnovenCapital,DebtFinancing,"250,000,000"
1,2,31/03/2021,Uniphore,Technology,Conversational Service Automation (CSA),PaloAlto,SorensonCapitalPartners,SeriesD,"140,000,000"
2,3,30/03/2021,Dunzo,E-commerce,Hyper-local delivery app,Bengaluru,KrishtalAdvisorsPteLtd,SeriesE,"8,000,000"
3,4,30/03/2021,BYJU’S,Edu-tech,Online tutoring,Bengaluru,"MCGlobalEdtech,BCapital,Baron,others",SeriesF,"460,000,000"
4,5,23/03/2021,SkilloVilla,Edu-tech,Career and job-oriented upskilling.,Bengaluru,"TitanCapital,others",Seed,"300,000,000"


# 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [7]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [13]:
try:
    
    # opening the trak page on automated chrome browser
    driver.get("https://digit.in/")
    driver.maximize_window()
    #close ad popup
    time.sleep(3)
    #click on Gaming option
    option=driver.find_element(By.XPATH,'/html/body/div[2]/div/ul/li[5]/a')
    option.click()
    time.sleep(3)
    #clcik on gaming laptop in list under gaming --i.e click on sub-option.
    g_mart=driver.find_element(By.XPATH,'/html/body/div[2]/div/ul/li[5]/div[2]/div/div[2]/div/ul[3]/li[4]/a')
    g_mart.click()
    #time.sleep(2)
    #scroll down
    driver.execute_script("window.scrollTo(0,800)")
except Exception as e:
    print(e)

In [479]:
#select the option gamimg Laptop
gam_lapu_option=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/ul/li[16]/span/a/img')
gam_lapu_option.click()

In [14]:
try:
    # for scraping all Product urls
    #scroll to pixel 800 of webpage
    prod_urls = []
    start=0
    end=1
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//div[@class="article_row"]/h2/a')
        for i in url:
            prod_urls.append(i.get_attribute("href"))
            #Scroll down  
        driver.execute_script("window.scrollTo(0,600)")
        time.sleep(2)

except Exception as e:
    print(e)

In [15]:
print((prod_urls))

['https://www.digit.in/top-products/best-asus-gaming-laptop-3912.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-rs-50-000-3715.html', 'https://www.digit.in/top-products/best-hp-gaming-laptop-3933.html', 'https://www.digit.in/top-products/best-dell-gaming-laptop-3935.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-60k-3980.html', 'https://www.digit.in/top-products/best-lenovo-gaming-laptops-in-india-3982.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-40k-3978.html', 'https://www.digit.in/top-products/best-msi-gaming-laptops-in-india-3983.html', 'https://www.digit.in/top-products/gaming-laptops-under-rs-70-000-3973.html', 'https://www.digit.in/top-products/best-amd-laptops-for-gaming-under-50000-4003.html']


In [62]:
name = []
desc = []
available = []
for url in prod_urls[0:1]:
    driver.get(url)
    time.sleep(10)
        
    driver.execute_script("window.scrollTo(0,1000)")
    
    nametags = driver.find_elements(By.XPATH,'//h3[@class="ga-not-fired"]//a')
    for i in nametags:
        nav = i.text
        name.append(nav)
     
    desctags = driver.find_elements(By.XPATH,'//div[@class="article_row big-image"]//p')
    for i in desctags:
        des = i.text
        desc.append(des)
     
    availabletags = driver.find_elements(By.XPATH,'//div[@class="companies_des"]')
    for i in availabletags:
        try:
            avail = i.text.split("\n")
            available.append(avail)
        except NoElementException as e:
            available.append('-')
            print(e)
    

In [63]:

name
desc
available
print(len(name),len(desc),len(available))
info =[name,desc]
info
available

6 6 3


[['Available', '₹ 69990', 'BUY NOW', 'Available', '₹ 79600', 'BUY NOW'],
 ['Available', '₹ 74990', 'BUY NOW'],
 ['Available', '₹ 86500', 'BUY NOW']]

In [64]:
df9 = pd.DataFrame({"Name":name,"Description":desc})
df9
 


,Name,Description
0,01. ASUS TUF Gaming,The ASUS TUF Gaming laptop is probably one of ...
1,02. ASUS Zephyrus G14,The ASUS Zephyrus G14 comes in various configu...
2,03. ASUS Vivobook Pro 14 OLED,What’s a good gaming laptop without a good dis...
3,04. ASUS ROG Strix G15,If an ASUS gaming laptop with a powerful CPU a...
4,05. ASUS TUF DASH F15,If you want high graphical power from a gaming...
5,06. ASUS ROG Flow X13,Can a gaming laptop also be a tablet? The answ...


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [14]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [16]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [17]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()
# opening the trak page on automated chrome browser
driver.get("http://www.forbes.com/")
driver.maximize_window()

In [18]:
try:
    #click on menu bar
    driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[1]').click()
    #click on option  Billionaires
    option = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]')
    option.click()
    time.sleep(3)
    
    driver.execute_script("window.scrollTo(0,1200)")
    suboption = driver.find_element(By.XPATH,'/html/body/div[1]/main/div/section/section[1]/div/div/div[1]/div/div[1]/div[1]/div[2]/a/h2')
    suboption.click()
    time.sleep(3)
    driver.execute_script("window.scrollTo(0,1800)")
except Exception as e:
    print(e)

In [21]:
Rank = []
Name = []
NetWorth = []
Age = []
Country = []
Source = []
Industry = []
leadge = driver.find_elements(By.XPATH,'//div[@class="table-row-group__container"]')
for i in leadge:
    #scraping ranks of billanier
    ranktags = driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for i in ranktags:
        r = i.text
        Rank.append(r)
    #sraping billanier names
    nametags = driver.find_elements(By.XPATH,'//*[@class="personName"]//div')
    for i in nametags:
        nav = i.text
        Name.append(nav)
    #for scraping networth
    net_worthtags = driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
    for i in net_worthtags:
        networth = i.text
        NetWorth.append(networth)
    #for scraping Age
    agetags = driver.find_elements(By.XPATH,'//div[@class="age"]')
    for i in agetags:
        age = i.text
        Age.append(age)
        
    #for scraping Country
    countrytags = driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for i in countrytags:
        country = i.text
        Country.append(age)
        
    #for scraping Source
    sourcetags = driver.find_elements(By.XPATH,'//span[@class="source-text"]')
    for i in sourcetags:
        source = i.text
        Source.append(source)
     #for scraping Industry
    industrytags= driver.find_elements(By.XPATH,'//span[@class="source-text"]')
    for i in industrytags:
        industry = i.text
        Industry.append(industry)
        

In [22]:
print(len(Rank),len(Name),len(NetWorth),len(Age),len(Country),len(Source),len(Industry))

2800 2800 2800 2800 2800 2800 2800


In [23]:
df_billanier = pd.DataFrame({'Rank':Rank[0:200],'Name':Name[0:200],'NetWorth':NetWorth[0:200],'Age':Age[0:200],'Country/Territory':Country[0:200],'Source':Source[0:200],'Industry':Industry[0:200]})
df_billanier

,Rank,Name,NetWorth,Age,Country/Territory,Source,Industry
0,1.,Elon Musk,$219 B,50,56,"Tesla, SpaceX","Tesla, SpaceX"
1,2.,Jeff Bezos,$171 B,58,56,Amazon,Amazon
2,3.,Bernard Arnault & family,$158 B,73,56,LVMH,LVMH
3,4.,Bill Gates,$129 B,66,56,Microsoft,Microsoft
4,5.,Warren Buffett,$118 B,91,56,Berkshire Hathaway,Berkshire Hathaway
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,56,beer,beer
196,197.,Leon Black,$10 B,70,56,private equity,private equity
197,197.,Joe Gebbia,$10 B,40,56,Airbnb,Airbnb
198,197.,David Geffen,$10 B,79,56,"movies, record labels","movies, record labels"


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [24]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [25]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [26]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()
# opening the trak page on automated chrome browser
driver.get("https://www.hostelworld.com/")
driver.maximize_window()

In [29]:
londontags = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
londontags.send_keys('London')

In [30]:
londonbutton = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')
londonbutton.click()
#driver.find_element(By.XPATH,'//button[@id="search-button"]').click()

In [31]:
driver.find_element(By.XPATH,'//button[@id="search-button"]').click()

In [124]:
H_name = []
Dist = []
Rating = []
Overall_reviews = []
Private_Price = []
Droms_Price = []
Facilities = []
Property_description = []

alldeatailtag=driver.find_elements(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[4]/div')
for i in alldeatailtag:
    #for Scraping Hostel Names
    hnametags = driver.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for j in hnametags: 
        H_name.append(j.text)
    #for Scraping Distance of hostel from city
    distags = driver.find_elements(By.XPATH,'//span[@class="description"]')
    for j in distags: 
        Dist.append(j.text.replace("Hostel - ",""))
    #for Scraping Ratings
    ratingtags = driver.find_elements(By.XPATH,'//div[@class="score orange big"]')
    for j in ratingtags: 
        Rating.append(j.text)
    #for Scraping Overall_reviews
    overalltags = driver.find_elements(By.XPATH,'//div[@class="reviews"]')
    for j in overalltags: 
        Overall_reviews.append(j.text.replace(" Total Reviews",""))
     #for Scraping  privates from price
    privatestags = driver.find_elements(By.XPATH,'//div[@class="price-col"]//div')
    for j in privatestags: 
        Private_Price.append(j.text)
        #for Scraping  Droms from price
    dromstags = driver.find_elements(By.XPATH,'//div[2][@class="price-col"]//div')
    for j in dromstags: 
        Droms_Price.append(j.text)
    #for Scraping  Facilities
    facilitiestags = driver.find_elements(By.XPATH,'//div[@class="rating-factors prop-card-tablet rating-factors small"]')
    for j in facilitiestags: 
        Facilities.append(j.text.replace("\n",","))
    #for Scraping property description
    property_descriptiontags = driver.find_elements(By.XPATH,'//div[@class="rating rating-summary-container big"]/div[2]/div[1]/span')
    for j in property_descriptiontags: 
        Property_description.append(j.text)
    
    
    

In [128]:
print(len(H_name), 
len(Dist),
len(Rating),
len(Overall_reviews),
len(Private_Price),
len(Droms_Price),
len(Facilities),
len(Property_description))

30 30 29 30 48 26 30 30


In [131]:
hostel_df = pd.DataFrame({"Hostel Name":H_name[0:26],"Distance from city to city":Dist[0:26],"Rating":Rating[0:26],"Overall_reviews":Overall_reviews[0:26],"Private_Price":Private_Price[0:26],"Facilities":Facilities[0:26],"Property_description":Property_description[0:26]})
hostel_df

,Hostel Name,Distance from city to city,Rating,Overall_reviews,Private_Price,Facilities,Property_description
0,Wombat's City Hostel London,3.6km from city centre,8.9,14466,Rs17748.96 Rs14199,"Perfect Location,Superb Staff,Fantastic Cleanl...",Fabulous
1,St Christopher's Village,1.8km from city centre,7.9,11806,Rs3522.6 Rs2818,"Perfect Location,Marvellous Staff,Excellent Cl...",Very Good
2,Hostel One Notting Hill,5.5km from city centre,9.4,1676,Rs1483.38 Rs1335,"Perfect Location,Superb Staff,Fantastic Cleanl...",Superb
3,Astor Hyde Park,4.3km from city centre,8.7,11832,Rs10168,"Perfect Location,Superb Staff,Fantastic Cleanl...",Fabulous
4,Generator London,3km from city centre,7.7,7306,Rs2881,"Perfect Location,Marvellous Staff,Excellent Cl...",Very Good
5,Safestay London Elephant & Castle,1.7km from city centre,7.3,4756,Rs1730,"Wonderful Location,Marvellous Staff,Excellent ...",Very Good
6,Urbany Hostel London,5.4km from city centre,9.1,594,Rs7660,"Perfect Location,Superb Staff,Fantastic Cleanl...",Superb
7,Pickwick Hall,2.3km from city centre,8.5,2545,Rs1976,"Perfect Location,Superb Staff,Excellent Cleanl...",Fabulous
8,St Christopher's Camden,4.3km from city centre,8.3,3948,Rs7184,"Perfect Location,Superb Staff,Excellent Cleanl...",Fabulous
9,Smart Russell Square Hostel,2.6km from city centre,7.1,9867,Rs1527,"Perfect Location,Brilliant Staff,Awesome Clean...",Very Good
